## Huntbook
# Tracing Covenant ShellCmd (via InstallUtil)

This interactive Kestrel huntbook is about tracing an adversary, that proxy executes code trough the InstallUtil, a command-line utility tool included in the . NET Framework.

The adversary uses it to install the "GruntHTTP.dll", a trojan malware and Covenant’s implant that is deployed to target system.


## Dataset Description

The dataset focusses on the proxy code execution via InstallUtil, a trusted Windows util.

Dataset: https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/defense_evasion/host/covenant_installutil.zip



## Adversary View

```
Upload Task: GruntHTTP.dll -> C:\ProgramData\GruntHTTP.dll
(wardog) > ShellCmd /shellcommand:"C:\Windows\Microsoft.NET\Framework64\v4.0.30319\InstallUtil.exe /logfile= /LogToConsole=false /u c:\ProgramData\GruntHTTP.dll"
```

Source: https://securitydatasets.com/notebooks/atomic/windows/execution/SDWIN-191027223020.html

## Tactics & Techniques

For the attack, the tactics [TA0005](https://attack.mitre.org/tactics/TA0005/), [TA0002](https://attack.mitre.org/tactics/TA0002/) and the techniques [T1218.004](https://attack.mitre.org/techniques/T1218/004/) are used.

The adversary uses Defense Evation, trying to avoid being detected and runs malicious code with Execution tactics. He uses System Binary Proxy Execution with the InstallUtil, a sub-technique of [T1218](https://attack.mitre.org/techniques/T1218/).

## Get Processes and Filter for "InstallUtil"

Typically the InstallUtil is used by developers to install and configure services or service applications. They can specify the '.exe' file of a Windows Service and install it onto the operating system. Here it is used to install malicious on the system.

Let's try to find all processes from the dataset, that use the "cmd.exe" to load the InstallUtil.

In [29]:
procs = GET process from https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/defense_evasion/host/covenant_installutil.zip 
where binary_ref.name LIKE 'cmd.exe' AND command_line LIKE '%InstallUtil%'

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-oca-asset*,x-oca-event*
procs,process,1,69,3,967,1088,8,4,667,28,180,640,640


We found records of one process entity. To examine it further, we display it.

In [30]:
DISP procs ATTR pid, x_unique_id, created, name, command_line 

sproc = procs

pid 
 x_unique_id 
 created 
 name 
 command_line 
 
 
 
 
 3776 
 {b34bc01c-add5-5f66-2010-000000000400} 
 2020-09-20T01:18:13.287Z 
 cmd.exe 
 "cmd.exe" /c C:\Windows\Microsoft.NET\Framework64\v4.0.30319\InstallUtil.exe /logfile= /LogToConsole=false /u c:\ProgramData\GruntHTTP.dll 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 artifact* 
 directory* 
 file* 
 ipv4-addr* 
 network-traffic* 
 process* 
 user-account* 
 windows-registry-key* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 sproc 
 process 
 1 
 69 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 *Number of related records cached.

## Analyze the Process
In the next hunting step, we analyze the suspicious process we found.

We can see, that the process with `pid` "3776" executes the `InstallUtil.exe` by checking the child processes. For that we check, which processes are created by the process with name `cmd.exe`.

In [31]:
cmd_proc = FIND process CREATED BY sproc

DISP cmd_proc

x_unique_id 
 created 
 name 
 command_line 
 id 
 pid 
 binary_ref.name 
 binary_ref.hashes.'SHA-1' 
 binary_ref.hashes.MD5 
 binary_ref.hashes.'SHA-256' 
 binary_ref.id 
 binary_ref.parent_directory_ref.path 
 binary_ref.parent_directory_ref.id 
 parent_ref.x_unique_id 
 parent_ref.created 
 parent_ref.name 
 parent_ref.command_line 
 parent_ref.id 
 parent_ref.pid 
 parent_ref.parent_ref.x_unique_id 
 parent_ref.parent_ref.created 
 parent_ref.parent_ref.name 
 parent_ref.parent_ref.command_line 
 parent_ref.parent_ref.id 
 parent_ref.parent_ref.pid 
 parent_ref.binary_ref.name 
 parent_ref.binary_ref.hashes.'SHA-1' 
 parent_ref.binary_ref.hashes.MD5 
 parent_ref.binary_ref.hashes.'SHA-256' 
 parent_ref.binary_ref.id 
 parent_ref.binary_ref.parent_directory_ref.path 
 parent_ref.binary_ref.parent_directory_ref.id 
 parent_ref.creator_user_ref.user_id 
 parent_ref.creator_user_ref.id 
 creator_user_ref.user_id 
 creator_user_ref.id 
 
 
 
 
 {b34bc01c-add5-5f66-2110-000000000400} 
 2020-09-20T01:18:13.294Z 
 conhost.exe 
 \??\C:\windows\system32\conhost.exe 0xffffffff -ForceV1 
 process--820ee083-891f-5a3b-a668-f81b94c0ed68 
 None 
 conhost.exe 
 11996F32DD85863A8C3BFF6D520F788A9211C8F7 
 C5E9B1D1103EDCEA2E408E9497A5A88F 
 BAF97B2A629723947539CFF84E896CD29565AB4BB68B0CEC515EB5C5D6637B69 
 file--4af5289a-dadf-532c-aa20-6eb439c763fa 
 C:\Windows\System32 
 directory--0a58d0c1-59e6-5afd-8252-dcd3f13e5622 
 {b34bc01c-add5-5f66-2010-000000000400} 
 2020-09-20T01:18:13.287Z 
 cmd.exe 
 "cmd.exe" /c C:\Windows\Microsoft.NET\Framework64\v4.0.30319\InstallUtil.exe /logfile= /LogToConsole=false /u c:\ProgramData\GruntHTTP.dll 
 process--d689999d-1ba7-5310-8af6-78edc64d9b28 
 3776 
 {b34bc01c-0266-5f65-540c-000000000400} 
 None 
 GruntHTTP.exe 
 "C:\Users\pgustavo\Desktop\GruntHTTP.exe" 
 process--1f79fffc-7c81-5349-9f86-c1b170168dde 
 5520 
 cmd.exe 
 None 
 None 
 None 
 file--bdb40b46-65f7-5a74-9516-60746576d4e7 
 C:\Windows\System32 
 directory--0a58d0c1-59e6-5afd-8252-dcd3f13e5622 
 THESHIRE\pgustavo 
 user-account--2dfb4c1a-ebd5-5c42-a8f1-26acde5c50eb 
 THESHIRE\pgustavo 
 user-account--2dfb4c1a-ebd5-5c42-a8f1-26acde5c50eb 
 
 
 {b34bc01c-add5-5f66-2210-000000000400} 
 2020-09-20T01:18:13.416Z 
 InstallUtil.exe 
 C:\Windows\Microsoft.NET\Framework64\v4.0.30319\InstallUtil.exe /logfile= /LogToConsole=false /u c:\ProgramData\GruntHTTP.dll 
 process--fb38481b-6881-53a8-9f2a-81afd9fbe548 
 None 
 InstallUtil.exe 
 BE9360E88838D064C2A13E8F154FD79D3B8BA0E1 
 4E481FD8FB0F3BDEE93ABF6D3C614EB7 
 9E8EB9A066BCC4DCC56AED56B73688922B5F921901938D8C02817D5390543585 
 file--5f42347e-eaae-5b04-bc2a-af9f6a6b0008 
 C:\Windows\Microsoft.NET\Framework64\v4.0.30319 
 directory--5c954cca-a5bd-5cc5-821f-226ae6154055 
 {b34bc01c-add5-5f66-2010-000000000400} 
 2020-09-20T01:18:13.287Z 
 cmd.exe 
 "cmd.exe" /c C:\Windows\Microsoft.NET\Framework64\v4.0.30319\InstallUtil.exe /logfile= /LogToConsole=false /u c:\ProgramData\GruntHTTP.dll 
 process--d689999d-1ba7-5310-8af6-78edc64d9b28 
 3776 
 {b34bc01c-0266-5f65-540c-000000000400} 
 None 
 GruntHTTP.exe 
 "C:\Users\pgustavo\Desktop\GruntHTTP.exe" 
 process--1f79fffc-7c81-5349-9f86-c1b170168dde 
 5520 
 cmd.exe 
 None 
 None 
 None 
 file--bdb40b46-65f7-5a74-9516-60746576d4e7 
 C:\Windows\System32 
 directory--0a58d0c1-59e6-5afd-8252-dcd3f13e5622 
 THESHIRE\pgustavo 
 user-account--2dfb4c1a-ebd5-5c42-a8f1-26acde5c50eb 
 THESHIRE\pgustavo 
 user-account--2dfb4c1a-ebd5-5c42-a8f1-26acde5c50eb 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 artifact* 
 directory* 
 file* 
 ipv4-addr* 
 network-traffic* 
 process* 
 user-account* 
 windows-registry-key* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 cmd_proc 
 process 
 2 
 586 
 3 
 967 
 1088 
 8 
 4 
 666 
 28 
 180 
 640 
 640 
 
 
 *Number of related records cached.

In the child processes we can see the process with name "InstallUtil.exe", that installs "GruntHTTP.dll". 

Next to the execution of the InstallUtil, we see that it also created a process with the name "conhost.exe", the Console Windows Host.

As a next step, we can analyze the user accounts that own the processes.

In [33]:
users = FIND user-account OWNED sproc

DISP users

user_id 
 id 
 
 
 
 
 THESHIRE\pgustavo 
 user-account--2dfb4c1a-ebd5-5c42-a8f1-26acde5c50eb 
 
 
 
 Block Executed in 3 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 artifact* 
 directory* 
 file* 
 ipv4-addr* 
 network-traffic* 
 process* 
 user-account* 
 windows-registry-key* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 users 
 user-account 
 1 
 25 
 3 
 967 
 1088 
 8 
 4 
 668 
 27 
 180 
 640 
 640 
 
 
 *Number of related records cached.

In [35]:
hosts = FIND x-oca-asset LINKED sproc
DISP hosts

hostname 
 id 
 
 
 
 
 WORKSTATION5.theshire.local 
 x-oca-asset--ce75297e-5c1e-5718-abb5-94740cb94591 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 artifact* 
 directory* 
 file* 
 ipv4-addr* 
 network-traffic* 
 process* 
 user-account* 
 windows-registry-key* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 hosts 
 x-oca-asset 
 1 
 640 
 3 
 967 
 1088 
 8 
 4 
 668 
 28 
 180 
 639 
 640 
 
 
 *Number of related records cached.

We see, that the machine with the name "WOKSTATION5.theshire" is linked to the process.

In [38]:
events = FIND x-oca-event LINKED sproc
DISP events

module,provider,code,action,category,id
Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon,1,Process Creation,"[""Process Create (rule: ProcessCreate)""]",x-oca-event--8a565f77-8c54-414f-bd0a-9c0b4effad56
Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon,1,Process Creation,"[""Process Create (rule: ProcessCreate)""]",x-oca-event--1d1ada4c-3c24-4d1a-a535-9a9332aed41a
Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon,1,Process Creation,"[""Process Create (rule: ProcessCreate)""]",x-oca-event--ca2eb397-9cea-47be-9eea-6b9c7dbc60f2
Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon,1,Process Creation,"[""Process Create (rule: ProcessCreate)""]",x-oca-event--1845988b-f584-4fe9-9f4b-6e870a943304
Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon,7,Image Loaded,"[""Image loaded (rule: ImageLoad)""]",x-oca-event--d2df9e1f-4f98-44c6-9072-eb3e3ad86298
Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon,7,Image Loaded,"[""Image loaded (rule: ImageLoad)""]",x-oca-event--0d5f0929-9480-464a-82cf-72e78ee1b747
Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon,7,Image Loaded,"[""Image loaded (rule: ImageLoad)""]",x-oca-event--26d078ff-bee3-4974-849c-23886bc93d5d
Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon,7,Image Loaded,"[""Image loaded (rule: ImageLoad)""]",x-oca-event--c5fb37a3-9e58-4ebb-b99c-07630cc78e61
Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon,13,Registry Value Set,"[""Registry value set (rule: RegistryEvent)""]",x-oca-event--44dc7cb7-f7b0-4f1b-8150-104ac6be7e08
Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon,1,Process Creation,"[""Process Create (rule: ProcessCreate)""]",x-oca-event--f4d6ec46-1b95-4538-a115-918b5ce3d828


With all this evidence, we can create an incident report.